<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=58c5de5eb80fe17d18a64e1ac5170104e89c13cd298b68c24bea8cb8bd9a4947
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-07 11:17:45
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -9.99 K (-0.07%)
Current PnL: -20.57 L (-13.65%)
CY Booked + Current PnL: -9.13 L (-6.06%)
-------------------
Total profit:  1.46 L
Total loss:  -22.03 L
-------------------
Total Booked + Current PnL: 17.85 L (14.43%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.11%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.07 L (62.46%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115402.0,14898.0,14206.0,0.27,14.82,12.31,28.96,79.0,1.05,0.82,53.83,MH,ATH,METALS
14,BLUESTARCO,2080.00,11.01,53.0,H-MC,2.99,191620.0,26950.0,16384.0,1.24,16.37,8.55,26.31,89.0,1.64,1.36,25.65,X40N,NTT,AC
77,TTKPRESTIG,770.00,93.38,39.0,M-SC,1.35,83336.0,-17441.0,17534.0,0.12,-17.31,21.04,0.09,245.0,-0.99,0.59,9.48,OX40N,NTT,DURABLES
19,CIPLA,1662.74,-22.18,44.0,H-LC,3.91,207733.0,3233.0,20067.0,0.21,1.58,9.66,11.39,10.0,0.16,1.48,10.67,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.66,50.0,H-LC,3.33,215670.0,5304.0,23486.0,0.55,2.52,10.89,13.68,37.0,0.23,1.54,19.46,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,STARHEALTH,761.0,14.16,64.0,H-SC,5.76,254102.0,-27021.0,159881.0,2.10,-9.61,62.92,47.26,144.0,-0.17,1.81,36.14,XY24,NTT,INSURANCE
85,WHIRLPOOL,2270.0,-46.47,34.0,M-SC,1.09,90548.0,-950.0,79700.0,2.03,-1.04,88.02,86.07,223.0,-0.01,0.64,31.49,XR,NTT,DURABLES
55,RAJOOENG,143.1,-39.13,47.0,H-SC,16.24,92290.0,-10210.0,50806.0,1.93,-9.96,55.05,39.61,114.0,-0.20,0.66,12.71,AR,ATH,MISC
44,JIOFIN,387.0,-1.39,53.0,H-LC,12.75,267847.0,482.0,64980.0,1.70,0.18,24.26,24.48,48.0,0.01,1.91,55.25,XY24,BTT,FINANCE
58,REPCOHOME,880.0,-60.51,51.0,H-SC,3.53,227609.0,-58896.0,316240.0,1.52,-20.56,138.94,89.82,134.0,-0.19,1.62,20.42,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1220.57,63.0,M-SC,20.83,179660.0,-1483.0,46945.0,-2.17,-0.82,26.13,25.10,235.0,-0.03,1.28,43.98,XY24,NTT,HEALTHCARE
38,INDIGOPNTS,1408.00,100.33,35.0,M-SC,2.12,131279.0,-43280.0,43309.0,-2.08,-24.79,32.99,0.02,221.0,-1.00,0.93,14.19,OX40N,NTT,PAINTS
69,SURYODAY,240.00,62.28,62.0,H-SC,6.67,148838.0,-30233.0,95956.0,-1.54,-16.88,64.47,36.71,135.0,-0.32,1.06,47.68,XR,NTT,BANKS
7,ATULAUTO,844.00,3554.05,48.0,M-SC,6.63,164016.0,-33116.0,122946.0,-1.31,-16.80,74.96,45.57,236.0,-0.27,1.17,16.28,XY24,NTT,AUTO
46,KPIGREEN,731.05,-7.49,31.0,H-SC,6.76,110212.0,-15085.0,74007.0,-1.28,-12.04,67.15,47.03,141.0,-0.20,0.78,38.00,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1220.57,63.0,M-SC,20.83,179660.0,-1483.0,46945.0,-2.17,-0.82,26.13,25.10,235.0,-0.03,1.28,43.98,XY24,NTT,HEALTHCARE
35,ICICIPRULI,790.0,-20.77,50.0,H-MC,2.05,136244.0,456.0,42290.0,0.40,0.34,31.04,31.48,107.0,0.01,0.97,12.50,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.25,44.0,M-LC,5.71,150319.0,-626.0,110079.0,0.13,-0.41,73.23,72.51,53.0,-0.01,1.07,5.58,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,66.0,H-SC,13.53,131544.0,-10935.0,30452.0,-0.35,-7.67,23.15,13.70,163.0,-0.36,0.94,55.26,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,55.0,H-SC,2.11,224640.0,-45027.0,81365.0,-0.55,-16.70,36.22,13.47,138.0,-0.55,1.60,14.22,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.38,39.0,M-SC,1.35,83336.0,-17441.0,17534.0,0.12,-17.31,21.04,0.09,245.0,-0.99,0.59,9.48,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.67,39.0,H-SC,1.51,141174.0,-34729.0,76756.0,0.24,-19.74,54.37,23.89,149.0,-0.45,1.01,21.92,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-25.22,40.0,H-MC,6.27,101270.0,-30565.0,72793.0,-1.19,-23.18,71.88,32.03,98.0,-0.42,0.72,13.43,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,51.0,M-MC,12.35,227321.0,2359.0,165285.0,-0.46,1.05,72.71,74.53,192.0,0.01,1.62,33.64,XY24,BTT,STEEL
32,HINDZINC,730.22,29.80,64.0,M-LC,6.19,211628.0,6552.0,106026.0,-1.02,3.19,50.10,54.89,52.0,0.06,1.51,28.06,X5K,ATH,METALS
14,BLUESTARCO,2080.00,11.01,53.0,H-MC,2.99,191620.0,26950.0,16384.0,1.24,16.37,8.55,26.31,89.0,1.64,1.36,25.65,X40N,NTT,AC
84,VOLTAS,1530.00,-2.06,49.0,H-MC,1.43,205500.0,13758.0,24002.0,0.24,7.18,11.68,19.69,99.0,0.57,1.46,14.46,XY25,NTT,AC
17,CAMS,5211.76,-5.16,49.0,H-SC,1.80,107946.0,5942.0,37986.0,0.78,5.83,35.19,43.06,122.0,0.16,0.77,23.94,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115402.0,14898.0,14206.0,0.27,14.82,12.31,28.96,79.0,1.05,0.82,53.83,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,71.0,H-MC,10.84,194818.0,15010.0,97409.0,0.50,8.35,50.00,62.52,88.0,0.15,1.39,45.28,XR,NTT,BANKS
32,HINDZINC,730.22,29.80,64.0,M-LC,6.19,211628.0,6552.0,106026.0,-1.02,3.19,50.10,54.89,52.0,0.06,1.51,28.06,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.83,35.0,H-SC,9.35,125997.0,2661.0,128971.0,0.05,2.16,102.36,106.72,119.0,0.02,0.90,26.04,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-46.47,34.0,M-SC,1.09,90548.0,-950.0,79700.0,2.03,-1.04,88.02,86.07,223.0,-0.01,0.64,31.49,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.30,30.0,H-LC,9.96,279922.0,-35920.0,148583.0,-0.05,-11.37,53.08,35.67,5.0,-0.24,1.99,0.91,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-17.90,33.0,H-LC,5.77,207187.0,-44581.0,97792.0,-0.02,-17.71,47.20,21.13,27.0,-0.46,1.48,11.45,X40,ATH,PAINTS
42,ITC,452.00,-39.19,42.0,H-LC,2.15,196040.0,-4098.0,24995.0,0.04,-2.05,12.75,10.44,4.0,-0.16,1.40,3.40,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-10.18,42.0,H-LC,8.01,252240.0,-4823.0,39955.0,-0.76,-1.88,15.84,13.67,24.0,-0.12,1.80,18.15,XY25,NTT,FMCG
74,TCS,4406.34,-30.29,43.0,H-LC,12.20,278428.0,-67528.0,135761.0,-0.88,-19.52,48.76,19.73,1.0,-0.50,1.98,2.55,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.17,54.0,H-LC,1.98,161010.0,-25085.0,72567.0,-0.97,-13.48,45.07,25.51,15.0,-0.35,1.15,18.65,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.19,42.0,H-LC,2.15,196040.0,-4098.0,24995.0,0.04,-2.05,12.75,10.44,4.0,-0.16,1.40,3.40,X40,NTT,FMCG
48,LTIM,7230.2,-4.34,54.0,H-LC,2.36,237150.0,-9001.0,88220.0,-0.08,-3.66,37.20,32.18,16.0,-0.10,1.69,31.07,X200,ATH,IT
51,NESTLEIND,1377.0,-10.51,50.0,H-LC,3.03,275348.0,9922.0,46864.0,-0.42,3.74,17.02,21.40,11.0,0.21,1.96,10.63,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.66,50.0,H-LC,3.33,215670.0,5304.0,23486.0,0.55,2.52,10.89,13.68,37.0,0.23,1.54,19.46,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.30,30.0,H-LC,9.96,279922.0,-35920.0,148583.0,-0.05,-11.37,53.08,35.67,5.0,-0.24,1.99,0.91,X40N,ATH,FMCG
74,TCS,4406.34,-30.29,43.0,H-LC,12.20,278428.0,-67528.0,135761.0,-0.88,-19.52,48.76,19.73,1.0,-0.50,1.98,2.55,X40,ATH,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.15,196040.0,-4098.0,24995.0,0.04,-2.05,12.75,10.44,4.0,-0.16,1.40,3.40,X40,NTT,FMCG
80,UNITDSPR,1644.00,-11.37,56.0,H-LC,6.88,232131.0,-2013.0,50628.0,-0.84,-0.86,21.81,20.77,86.0,-0.04,1.65,6.00,X40N,NTT,BREWERIES
30,HCLTECH,1937.36,-1.51,48.0,H-LC,9.87,224541.0,-17383.0,79622.0,0.88,-7.19,35.46,25.73,8.0,-0.22,1.60,6.11,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.17,54.0,H-LC,1.98,161010.0,-25085.0,72567.0,-0.97,-13.48,45.07,25.51,15.0,-0.35,1.15,18.65,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,42.0,H-LC,2.15,196040.0,-4098.0,24995.0,0.04,-2.05,12.75,10.44,4.0,-0.16,1.40,3.40,X40,NTT,FMCG
19,CIPLA,1662.74,-22.18,44.0,H-LC,3.91,207733.0,3233.0,20067.0,0.21,1.58,9.66,11.39,10.0,0.16,1.48,10.67,X40N,ATH,PHARMA
5,ASIANPAINT,3465.66,-17.90,33.0,H-LC,5.77,207187.0,-44581.0,97792.0,-0.02,-17.71,47.20,21.13,27.0,-0.46,1.48,11.45,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,50.0,H-LC,3.33,215670.0,5304.0,23486.0,0.55,2.52,10.89,13.68,37.0,0.23,1.54,19.46,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-30.29,43.0,H-LC,12.20,278428.0,-67528.0,135761.0,-0.88,-19.52,48.76,19.73,1.0,-0.50,1.98,2.55,X40,ATH,IT
40,INFY,2275.00,-19.96,45.0,H-LC,8.09,311619.0,-1269.0,172949.0,-0.88,-0.41,55.50,54.87,3.0,-0.01,2.22,6.22,X40,BTT,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.15,196040.0,-4098.0,24995.0,0.04,-2.05,12.75,10.44,4.0,-0.16,1.40,3.40,X40,NTT,FMCG
83,VBL,671.64,-22.30,30.0,H-LC,9.96,279922.0,-35920.0,148583.0,-0.05,-11.37,53.08,35.67,5.0,-0.24,1.99,0.91,X40N,ATH,FMCG
1,ABB,7934.00,-39.18,56.0,H-LC,7.49,254184.0,-7435.0,126660.0,1.49,-2.84,49.83,45.57,7.0,-0.06,1.81,8.16,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,31.65,79565.0,-33984.0,74035.0,0.51,-29.93,93.05,35.27,268.0,-0.46,0.57,94.74,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,45.0,H-SC,11.45,91710.0,-6270.0,27871.0,-0.80,-6.40,30.39,22.05,152.0,-0.22,0.65,33.31,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115402.0,14898.0,14206.0,0.27,14.82,12.31,28.96,79.0,1.05,0.82,53.83,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,66.0,H-SC,13.53,131544.0,-10935.0,30452.0,-0.35,-7.67,23.15,13.70,163.0,-0.36,0.94,55.26,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.55,145755.0,-3465.0,147650.0,1.26,-2.32,101.30,96.62,208.0,-0.02,1.04,57.74,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.55,145755.0,-3465.0,147650.0,1.26,-2.32,101.30,96.62,208.0,-0.02,1.04,57.74,XY25,NTT,FINANCE
69,SURYODAY,240.00,62.28,62.0,H-SC,6.67,148838.0,-30233.0,95956.0,-1.54,-16.88,64.47,36.71,135.0,-0.32,1.06,47.68,XR,NTT,BANKS
11,BANKINDIA,190.00,-24.22,71.0,H-MC,10.84,194818.0,15010.0,97409.0,0.50,8.35,50.00,62.52,88.0,0.15,1.39,45.28,XR,NTT,BANKS
26,GREENPANEL,537.00,228.81,58.0,M-SC,3.65,146937.0,-36141.0,115125.0,-0.36,-19.74,78.35,43.14,230.0,-0.31,1.05,38.21,XY24,NTT,MISC
32,HINDZINC,730.22,29.80,64.0,M-LC,6.19,211628.0,6552.0,106026.0,-1.02,3.19,50.10,54.89,52.0,0.06,1.51,28.06,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.1
1,25,43.3
2,50,75.0


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.07
LC    32.21
MC    23.75
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.94
X40      14.60
X40N     11.20
XR       11.00
XY25     10.41
AR        8.23
OX40N     7.70
X200      1.69
MH        1.60
X5K       1.51
SR        1.15
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.00
H-LC    25.66
H-MC    20.52
M-SC    13.65
M-LC     5.53
M-MC     2.93
L-SC     1.42
L-LC     1.02
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.76,-6.24,41.26
IT,12.62,-18.66,82.93
FINANCE,10.48,-14.57,69.89
MISC,6.95,-17.79,83.06
BANKS,6.39,-11.39,69.99
PAINTS,5.55,-18.47,36.13
ELECTRICAL,5.39,-8.81,46.98
INSURANCE,3.80,-3.62,42.35
AUTO,3.52,-16.46,72.14


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3232683.0,22
XR,1365081.0,14
AR,1175092.0,9
X40,790758.0,10
X40N,659886.0,9
OX40N,563458.0,10
XY25,476968.0,7
SR,261069.0,2
X5K,106026.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3720391.0,29
M-SC,1514216.0,17
H-MC,1271052.0,15
H-LC,1193129.0,15
M-LC,374490.0,4
M-MC,371743.0,2
L-SC,261376.0,3
L-MC,59902.0,1
L-LC,41155.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1212539.0      6
M-SC       XY24       854617.0      7
H-SC       AR         807661.0      5
           XR         797469.0      7
H-MC       XY24       570419.0      4
H-LC       X40        511119.0      5
M-MC       XY24       371743.0      2
H-SC       X40N       309280.0      4
           SR         261069.0      2
           OX40N      258366.0      4
H-LC       X40N       219278.0      3
H-MC       X40        207039.0      4
H-LC       AR         199227.0      2
H-MC       XY25       177858.0      2
L-SC       XR         172138.0      2
M-SC       AR         168204.0      2
M-LC       XY24       158385.0      2
M-SC       XY25       147650.0      1
           OX40N      143061.0      4
H-MC       X40N       131328.0      2
M-SC       XR         128084.0      2
H-LC       XY25       110305.0      3
M-LC       XR         110079.0      1
           X5K        106026.0      1
H-MC       XR          97409.0      1
L-SC       OX40N       89238.0      1
H-LC       X200        88220.0      1
H-SC       MH          74007.0      1
H-MC       OX40N       72793.0      1
M-SC       X40         72600.0      1
H-LC       XY24        64980.0      1
L-MC       XR          59902.0      1
L-LC       XY25        41155.0      1
H-MC       MH          14206.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
